In [2]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

import matplotlib.pyplot as plt

pass

In [3]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)
names.sample(5)

,sexe,preusuel,annais,dpt,nombre
1935318,2,ANNIE,1957,83,48
600940,1,GASPARD,2005,91,6
808889,1,JEAN-LOUP,1952,87,3
2457352,2,FLORENCE,1949,68,3
719697,1,HUGUES,1960,54,5


In [4]:
names_fem = names[names.sexe==2]
names_masc = names[names.sexe==1]
names_fem

,sexe,preusuel,annais,dpt,nombre
1741138,2,AALIYA,2017,75,3
1741140,2,AALIYAH,2001,92,4
1741141,2,AALIYAH,2001,971,5
1741142,2,AALIYAH,2002,06,3
1741143,2,AALIYAH,2002,13,3
...,...,...,...,...,...
3727545,2,ZYA,2013,44,4
3727546,2,ZYA,2013,59,3
3727547,2,ZYA,2017,974,3
3727548,2,ZYA,2018,59,3


In [5]:
names_fem_popular = names_fem[['preusuel', 'nombre']].groupby('preusuel', as_index=False).sum()
top_names_fem = names_fem_popular.sort_values('nombre', ascending=False)[:20]

names_masc_popular = names_masc[['preusuel', 'nombre']].groupby('preusuel', as_index=False).sum()
top_names_masc = names_masc_popular.sort_values('nombre', ascending=False)[:20]

names_popular = names[['preusuel', 'nombre']].groupby('preusuel', as_index=False).sum()
top_names = names_popular.sort_values('nombre', ascending=False)[:20]

print(list(top_names_fem['preusuel']))
print(list(top_names_masc['preusuel']))
print(list(top_names['preusuel']))


['MARIE', 'JEANNE', 'FRANÇOISE', 'MONIQUE', 'CATHERINE', 'NATHALIE', 'ISABELLE', 'JACQUELINE', 'ANNE', 'SYLVIE', 'MARTINE', 'MADELEINE', 'NICOLE', 'SUZANNE', 'HÉLÈNE', 'CHRISTINE', 'LOUISE', 'MARGUERITE', 'DENISE', 'CHRISTIANE']
['JEAN', 'PIERRE', 'MICHEL', 'ANDRÉ', 'PHILIPPE', 'LOUIS', 'RENÉ', 'ALAIN', 'JACQUES', 'BERNARD', 'MARCEL', 'DANIEL', 'ROGER', 'PAUL', 'ROBERT', 'CLAUDE', 'HENRI', 'CHRISTIAN', 'GEORGES', 'NICOLAS']
['MARIE', 'JEAN', 'PIERRE', 'MICHEL', 'ANDRÉ', 'JEANNE', 'PHILIPPE', 'LOUIS', 'RENÉ', 'ALAIN', 'JACQUES', 'BERNARD', 'MARCEL', 'CLAUDE', 'DANIEL', 'ROGER', 'PAUL', 'ROBERT', 'DOMINIQUE', 'GEORGES']


In [6]:
names_masc_filt = names_masc[names_masc['preusuel'].isin(list(top_names_masc['preusuel']))].groupby(['preusuel', 'annais'], as_index=False).sum().drop('sexe', axis=1)
names_fem_filt = names_fem[names_fem['preusuel'].isin(list(top_names_fem['preusuel']))].groupby(['preusuel', 'annais'], as_index=False).sum().drop('sexe', axis=1)
names_filt = names[names['preusuel'].isin(list(top_names['preusuel']))].groupby(['preusuel', 'annais'], as_index=False).sum().drop('sexe', axis=1)

print(names_masc_filt.head())
print(names_fem_filt.head())
print(names_filt.head())

  preusuel annais  nombre
0    ALAIN   1900      83
1    ALAIN   1901      99
2    ALAIN   1902     106
3    ALAIN   1903     120
4    ALAIN   1904     136
  preusuel annais  nombre
0     ANNE   1900    3440
1     ANNE   1901    3455
2     ANNE   1902    3721
3     ANNE   1903    3744
4     ANNE   1904    3552
  preusuel annais  nombre
0    ALAIN   1900      83
1    ALAIN   1901      99
2    ALAIN   1902     106
3    ALAIN   1903     120
4    ALAIN   1904     136


In [12]:

selection = alt.selection_multi(fields=['preusuel'], bind='legend')
chart = alt.Chart(names_filt).mark_line().encode(
  x='annais:T',
  y='nombre:Q',
  color=alt.condition(selection, 'preusuel:N', alt.value('lightgray')),
  size=alt.condition(selection, alt.value(3), alt.value(1))
).add_selection(selection).properties(
    width=500,
    height=400,
    title='Top 20 prénoms (masculins et féminins confondus)'
)
chart

alt.Chart(...)

Cliquer sur un prénom en légende sur la visualisation ci-dessus

In [13]:
selection = alt.selection_multi(fields=['preusuel'], bind='legend')
chart = alt.Chart(names_masc_filt).mark_line().encode(
  x='annais:T',
  y='nombre:Q',
  color=alt.condition(selection, 'preusuel:N', alt.value('lightgray')),
  size=alt.condition(selection, alt.value(3), alt.value(1))
).add_selection(selection).properties(
    width=500,
    height=400,
    title='Top 20 prénoms (masculins uniquement)'
)
chart

chart

alt.Chart(...)

Si on clique dans la légende sur le prénom ""

In [14]:
selection = alt.selection_multi(fields=['preusuel'], bind='legend')
chart = alt.Chart(names_fem_filt).mark_line().encode(
  x='annais:T',
  y='nombre:Q',
  color=alt.condition(selection, 'preusuel:N', alt.value('lightgray')),
  size=alt.condition(selection, alt.value(3), alt.value(1))
).add_selection(selection).properties(
    width=500,
    height=400,
    title='Top 20 prénoms (féminins uniquement)'
)
chart

alt.Chart(...)

Si on clique dans la légende sur le prénom "Natalie", on peut apercevoir qu'il connaît un pic de popularité dans les années 1970

Notre première visualisation est un graphique de courbes chart créé avec altair. Il représente l’évolution des 20 noms les plus utilisés au cours du temps. Nous en avons fait plusieurs versions : une version pour les sexes confondus, une autre pour les prénoms masculins et enfin une dernière pour les prénoms féminin.

Au niveau des 20 noms les plus utilisés, nous pouvons clairement constater certaines évolutions. De manière générale, aucuns noms (féminins ou masculins) n’arrivent à garder une popularité constate. Nous pouvons apercevoir que certains noms voient leur nombre décroître très fortement au fil des années, et d’autres qui deviennent populaires très rapidement pendant un court moment. Nous pouvons aussi constater des tendances générales, par exemple la baisse générale des attributions de tous les noms  dans les années 1918 (faible taux de natalité lié à la Première Guerre Mondiale ?)

Notre visualisation ici nous permet de suivre la tendance d’un prénom du top 20 en particulier. En cliquant sur un prénom en légende, la courbe lui correspondant va venir se démarquer des autres, nous permettant ainsi de pleinement l’apprécier à travers u nuage de courbes. 

Dans cette visualisation, il aurait été intéressant de mettre en lumière les noms les plus attribués par fourchette d’années (par exemple les noms les plus attribués de 1900 à 1940, de 1940 à 1980, etc…). En effet, la plupart des prénoms retenus dans le graphique ont connu un pic de popularité entre 1920 et 1980, d’où leur nombre très élevé. Cependant, les prénoms tendent à se diversifier au fil des années. Cette visualisation pourrait donc bénéficier d’une mise en valeur des « nouveaux » prénoms les plus populaires à partir des années 1980 par exemple (l’année où la majorité des prénoms populaires d’avant se retrouvent en dessous du nombre de 5000)
